In [1]:
import os
import warnings
import json
import numpy as np
import pandas as pd
import polar_diagrams
from sklearn.datasets import load_wine

from dash import Dash, dcc, html, Input, Output, callback, State, ctx, Patch
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
import plotly.graph_objects as go

In [2]:
df_wine_data = load_wine(return_X_y=False, as_frame=True)['data']
df_wine_data['od_diluted'] = df_wine_data['od280/od315_of_diluted_wines']
df_wine_data.drop(['od280/od315_of_diluted_wines', 'proline'], axis=1,
                  inplace=True)
df_wine_data

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od_diluted
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93
...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62


In [3]:
def app_create_dashboard(df_input, string_reference_model,
                         string_diagram_type='taylor',
                         string_mid_type='normalized'):
    dash_app = Dash("Polar Diagrams Dashboard",
                    external_stylesheets=[dbc.themes.BOOTSTRAP],
                    meta_tags=[{"name": "viewport",
                                "content": "width=device-width"}],)
    dash_app.title = "Polar Diagrams Dashboard"
    dash_app.css.config.serve_locally = True
    dash_app.scripts.config.serve_locally = True
    _INT_CHART_WIDTH = 1400
    _INT_CHART_HEIGHT = 500
    _INT_NUM_OF_MODELS = len(df_input.columns)

    list_valid_diagram_types = ['taylor', 'mid']
    list_valid_mid_types = ['scaled', 'normalized']

    if string_diagram_type not in list_valid_diagram_types:
        raise ValueError('string_diagram_type not in ' +
                         str(list_valid_diagram_types))

    if string_diagram_type == 'mid' and (
            string_mid_type not in list_valid_mid_types):
        raise ValueError('string_mid_type not in ' +
                         str(list_valid_mid_types))

    list_warning_caught = None
    warnings.formatwarning = lambda msg, *args, **kwargs: str(msg)

    if string_diagram_type == 'mid':
        with warnings.catch_warnings(record=True) as warning_tmp:
            # Cause all warnings to always be triggered.
            warnings.simplefilter("default")

            chart_left = polar_diagrams.chart_create_mi_diagram(
                df_input, string_reference_model=string_reference_model,
                string_mid_type=string_mid_type).update_layout(
                dragmode='zoom', clickmode='event+select', hovermode=False,
                width=round(_INT_CHART_WIDTH/2.6),
                height=_INT_CHART_HEIGHT + 40*round(_INT_NUM_OF_MODELS/4),
                margin={'l':40, 'r':40})
            chart_right = polar_diagrams.chart_create_mi_diagram(
                df_input, string_reference_model=string_reference_model,
                string_mid_type=string_mid_type).update_layout(
                dragmode='select', clickmode='event+select',
                width=int(_INT_CHART_WIDTH/1.3),
                height=_INT_CHART_HEIGHT*1.3,
                margin={'l':150, 'r':40})

            list_warning_caught = warning_tmp
    else:
        with warnings.catch_warnings(record=True) as warning_tmp:
            # Cause all warnings to always be triggered.
            warnings.simplefilter("default")

            chart_left = polar_diagrams.chart_create_taylor_diagram(
                df_input,
                string_reference_model=string_reference_model).update_layout(
                dragmode='zoom', clickmode='event+select', hovermode=False,
                width=round(_INT_CHART_WIDTH/2.6),
                height=_INT_CHART_HEIGHT + 40*round(_INT_NUM_OF_MODELS/4),
                margin={'l':40, 'r':40})
            chart_right = polar_diagrams.chart_create_taylor_diagram(
                df_input,
                string_reference_model=string_reference_model).update_layout(
                dragmode='select', clickmode='event+select',
                width=int(_INT_CHART_WIDTH/1.3),
                height=_INT_CHART_HEIGHT*1.3,
                margin={'l':150, 'r':40})

            list_warning_caught = warning_tmp

    string_warnings = ''
    string_one_warning = ''
    for warning_tmp in list_warning_caught:
        if 'RuntimeWarning' in warnings.formatwarning(warning_tmp):
            string_one_warning = warnings.formatwarning(
                warning_tmp)[11:208].replace('\\n', ' ')
            if string_one_warning in string_warnings:
                continue
            else:
                string_warnings += string_one_warning

    # We disable a legend for the second diagram by traversing traces
    dict_right = chart_right.to_dict()
    for int_i in range(len(dict_right['data'])):
        dict_right['data'][int_i]['showlegend'] = False
    chart_right = go.Figure(dict_right)

    dict_left = chart_left.to_dict()
    global _FLOAT_MAX_R
    _FLOAT_MAX_R = dict_left['layout']['polar']['radialaxis']['range'][1]
    global _INT_MAX_THETA
    _INT_MAX_THETA = dict_left['layout']['polar']['angularaxis'][
        'tickvals'][0]

    # ====================================================
    # TODO: Create a nicer layout with a title, logo, etc.
    # ====================================================
    # TODO: Add a box for displaying errors from the polar-diagrams library
    # TODO: This is important if we have overlapping models
    # ====================================================

    dash_app.layout = dbc.Container(
        [
            dbc.Row(
                dbc.Col(
                    html.Div(
                        html.H1("Polar Diagrams Dashboard"),
                        style={"font-family": 'open sans'}),
                    width=True)),
            dbc.Row(
                [
                    dbc.Col(
                        dcc.Graph(
                            id="chart-left",
                            figure=chart_left,
                            config={
                                'modeBarButtonsToRemove': ['select', 'pan', 'lasso',
                                                           'zoomIn', 'zoomOut',
                                                           'autoScale', 'resetScale'],
                                'staticPlot': False,
                                'displaylogo': False,
                                'showAxisDragHandles': False}),
                        width=4,
                        align='center',
                        style={'border': '1px solid'}),
                    dbc.Col(
                        dcc.Graph(
                            id="chart-right",
                            figure=chart_right,
                            config={
                                'modeBarButtonsToRemove': ['zoom', 'pan', 'lasso',
                                                           'zoomIn', 'zoomOut',
                                                           'select', 'autoScale',
                                                           'resetScale'],
                                'displaylogo': False,
                                'showAxisDragHandles': False}),
                        width=True,
                        align='center',
                        style={'border': '1px solid'})
                ],
                className="g-0",
                justify="center",
            ),
            dbc.Row(
                dbc.Col(
                    html.Div(
                        dbc.Alert(
                            string_warnings,
                            color="warning",
                            is_open=True if string_warnings != '' else False)),
                    width=True),
                    style={'border': '1px solid'}),
        ],
        fluid=True)

    '''
    dash_app.layout = html.Div(
        [
            html.Div(html.H1("Polar Diagrams Dashboard"),
                     style={"font-family": 'open sans'}),
            html.Div(
                dcc.Graph(
                    id="chart-left",
                    figure=chart_left,
                    config={
                        'modeBarButtonsToRemove': ['select', 'pan', 'lasso',
                                                   'zoomIn', 'zoomOut',
                                                   'autoScale', 'resetScale'],
                        'staticPlot': False,
                        'displaylogo': False,
                        'showAxisDragHandles': False}),
                className="two-columns",
                style={'width': '30%','display': 'inline-block'},
            ),
            html.Div(
                dcc.Graph(
                    id="chart-right",
                    figure=chart_right,
                    config={
                        'modeBarButtonsToRemove': ['zoom', 'pan', 'lasso',
                                                   'zoomIn', 'zoomOut',
                                                   'select', 'autoScale',
                                                   'resetScale'],
                        'displaylogo': False,
                        'showAxisDragHandles': False}),
                className="two-columns",
                style={'width': '70%','display': 'inline-block'},
            ),
        ],
        className="row",
    )
    '''

    dash_app.run(debug=True, jupyter_mode='external')

    return None


@callback(
    Output(component_id="chart-left", component_property="figure", allow_duplicate=True),
    Output(component_id="chart-right", component_property="figure", allow_duplicate=True),
    Input(component_id="chart-left", component_property="restyleData"),
    State('chart-left', 'figure'),
    State('chart-right', 'figure'),
    prevent_initial_call=True,
)
def list_update_legends(list_legend_points, dict_left, dict_right):
    # ====================================================
    # TODO: Combine the two callbacks by using the following context property
    # TODO: list(ctx.triggered_prop_ids.keys())[0].split('.')[1]
    # TODO: This will either return restyleData or relayoutData
    # ====================================================
    chart_left_updated = Patch()
    chart_right_updated = Patch()
    if list_legend_points:
        # Legend click gives the following output
        # [{"visible": ["legendonly"]}, [10]]
        # [{"visible": [true]}, [1]]

        for int_i, int_legend_point in enumerate(list_legend_points[1]):
            for int_j, dict_one_trace in enumerate(dict_right['data']):
                if dict_one_trace['name'].startswith(
                        str(int_legend_point) + '.'):
                    if isinstance(
                        list_legend_points[0]['visible'][int_i], bool) and (
                        list_legend_points[0]['visible'][int_i], bool == True):
                        chart_right_updated['data'][int_j][
                            'visible'] = True
                    else:
                        chart_right_updated['data'][int_j][
                            'visible'] = False
    else:
        raise PreventUpdate

    return chart_left_updated, chart_right_updated


@callback(
    Output(component_id="chart-left", component_property="figure"),
    Output(component_id="chart-right", component_property="figure"),
    Input(component_id="chart-left", component_property="relayoutData"),
    State('chart-left', 'figure'),
    State('chart-right', 'figure'),
    prevent_initial_call=True
)
def list_update_zooms(dict_selected_range, dict_left,
                      dict_right):

    chart_left_updated = Patch()
    chart_right_updated = Patch()

    if dict_selected_range and (
            'polar.radialaxis.range' in dict_selected_range):

        dict_radial_range = dict_selected_range['polar.radialaxis.range']

        for int_i, trace in enumerate(dict_left['data']):
            if 'name' in trace and trace['name'] == 'Selection':
                del chart_left_updated['data'][int_i]

        # Here we check if double click was not detected. If it was detected
        # we just had to remove the Selection trace, which we did above.
        # If it was not detected, that means we have to create a new Selection
        # {  'polar.angularaxis.rotation': 0,
        #    'polar.radialaxis.angle': 0,
        #    'polar.radialaxis.range': [0, 16.353330541878254]
        # }
        if 'polar.angularaxis.rotation' not in dict_selected_range and (
                'polar.radialaxis.angle' not in dict_selected_range):

            # We create a circular rectangle of 60 points by creating them and
            # connecting them with a line
            np_alpha = np.linspace(0, _INT_MAX_THETA, 60).tolist()
            np_selection_theta = np_alpha + np_alpha[::-1] + [np_alpha[0]]

            chart_left_updated['data'].append(
                go.Scatterpolar(r=[dict_radial_range[0]]*60 +\
                                  [dict_radial_range[1]]*60 +\
                                  [dict_radial_range[0]],
                                theta =np_selection_theta,
                                name='Selection',
                                fill='toself',
                                mode='lines',
                                showlegend=False,
                                line=dict(
                                    color='lightgrey',
                                    dash='dot',
                                    width=3)))

        chart_left_updated['layout']['polar']["radialaxis"][
            "autorange"] = False
        chart_left_updated['layout']['polar']["radialaxis"][
            'rangemode'] = 'normal'
        chart_right_updated['layout']['polar']["radialaxis"][
            "autorange"] = False
        chart_right_updated['layout']['polar']["radialaxis"][
            'rangemode'] = 'normal'

        chart_left_updated['layout']['polar']["radialaxis"][
            "range"] = [0, _FLOAT_MAX_R]
        chart_right_updated['layout']['polar']["radialaxis"][
            "range"] = [dict_radial_range[0], dict_radial_range[1]]

    else:
        raise PreventUpdate

    return chart_left_updated, chart_right_updated


app_create_dashboard(df_wine_data, 'alcohol', 'mid', 'scaled')

Dash app running on http://127.0.0.1:8050/
